## 1. Import Libraries

In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


## 2. Load Dataset

In [2]:
df = pd.read_csv("water_consumption.csv", delimiter="\t")
df.head()


,initial,final,price,month,year,block,apartment
0,3535,3565,464.1,7,2023,A,11
1,3375,3402,380.1,7,2023,A,12
2,3620,3651,492.1,7,2023,A,21
3,4681,4707,352.1,7,2023,A,22
4,2400,2425,324.1,7,2023,A,31


## 3. Creation of Consumption Column and Dummies

In [3]:
df["consumption"] = df["final"] - df["initial"]
df = df.drop(columns=["month", "year"])
df = pd.get_dummies(df, columns=["block"])
df.head()


,initial,final,price,apartment,consumption,block_A,block_B,block_C,block_D,block_E,block_F,block_G
0,3535,3565,464.1,11,30,True,False,False,False,False,False,False
1,3375,3402,380.1,12,27,True,False,False,False,False,False,False
2,3620,3651,492.1,21,31,True,False,False,False,False,False,False
3,4681,4707,352.1,22,26,True,False,False,False,False,False,False
4,2400,2425,324.1,31,25,True,False,False,False,False,False,False


## 4. OLS Summary

In [4]:
y = df["consumption"]
x = df.drop(columns=["consumption"])

x = x.apply(pd.to_numeric, errors="coerce").astype(float)
x = sm.add_constant(x, has_constant="add")

model = sm.OLS(y, x).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:            consumption   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.390e+28
Date:                Sat, 23 Aug 2025   Prob (F-statistic):               0.00
Time:                        20:30:29   Log-Likelihood:                 10562.
No. Observations:                 390   AIC:                        -2.110e+04
Df Residuals:                     379   BIC:                        -2.106e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.751e-13    7.2e-14     -7.982      0.0

## 5. Select Significant Variables

In [5]:
significant_vars = model.pvalues[model.pvalues <= 0.5].index.tolist()
significant_vars = [v for v in significant_vars if v != "const"]

print(significant_vars)


['initial', 'final', 'apartment', 'block_A', 'block_B', 'block_C', 'block_D', 'block_E', 'block_F', 'block_G']


## 6. Individual Regressions

In [6]:
df_sig = df[significant_vars + ["consumption"]]

results = []
for col in significant_vars:
    x = sm.add_constant(df_sig[[col]].astype(float))
    y = df_sig["consumption"].astype(float)
    model = sm.OLS(y, x).fit()
    results.append({
        "Variable": col,
        "P-value": model.pvalues[col],
        "R² (individual)": model.rsquared
    })

summary_table = pd.DataFrame(results).sort_values(by="R² (individual)", ascending=False)
print(summary_table)


    Variable       P-value  R² (individual)
1      final  2.578461e-24         0.234433
0    initial  1.693518e-23         0.227032
3    block_A  5.549339e-04         0.030296
4    block_B  6.160288e-02         0.008975
7    block_E  6.175288e-02         0.008965
6    block_D  8.460062e-02         0.007646
8    block_F  1.765181e-01         0.004703
9    block_G  4.038639e-01         0.001797
2  apartment  5.054138e-01         0.001144
5    block_C  6.058566e-01         0.000687


## 7. Simple Linear Regression

In [7]:
for var in summary_table["Variable"]:
  print("Variable chosen: " + var )
  x = df[[var]]
  y = df["consumption"]

  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=21)

  results_poly = []

  for degree in range(1, 8):
      poly_f = PolynomialFeatures(degree=degree)
      x_train_poly = poly_f.fit_transform(x_train)
      x_test_poly = poly_f.transform(x_test)

      lin_reg = LinearRegression()
      lin_reg.fit(x_train_poly, y_train)

      r2_train = lin_reg.score(x_train_poly, y_train)
      r2_test = lin_reg.score(x_test_poly, y_test)
      diff = abs(r2_train - r2_test)

      results_poly.append((degree, r2_train, r2_test, diff))
      print(f"Degree: {degree} — R² Train: {r2_train:.4f}, R² Test: {r2_test:.4f}, Diff: {diff:.4f}")

  results_df = pd.DataFrame(results_poly, columns=["Degree", "R2_Train", "R2_Test", "Difference"])
  print("\nBest degree (smallest train-test R² difference):")
  print(results_df.loc[results_df["Difference"].idxmin()])
  print("\n-----------------------------------------------------------\n")


Variable chosen: final
Degree: 1 — R² Train: 0.2588, R² Test: 0.1580, Diff: 0.1008
Degree: 2 — R² Train: 0.2708, R² Test: 0.1250, Diff: 0.1458
Degree: 3 — R² Train: 0.2796, R² Test: 0.1676, Diff: 0.1120
Degree: 4 — R² Train: 0.2807, R² Test: 0.1327, Diff: 0.1480
Degree: 5 — R² Train: 0.2802, R² Test: -0.0206, Diff: 0.3008
Degree: 6 — R² Train: 0.2799, R² Test: -0.3378, Diff: 0.6177
Degree: 7 — R² Train: 0.2791, R² Test: -2.0334, Diff: 2.3126

Best degree (smallest train-test R² difference):
Degree        1.000000
R2_Train      0.258764
R2_Test       0.157978
Difference    0.100786
Name: 0, dtype: float64

-----------------------------------------------------------

Variable chosen: initial
Degree: 1 — R² Train: 0.2509, R² Test: 0.1518, Diff: 0.0991
Degree: 2 — R² Train: 0.2631, R² Test: 0.1197, Diff: 0.1434
Degree: 3 — R² Train: 0.2720, R² Test: 0.1622, Diff: 0.1097
Degree: 4 — R² Train: 0.2733, R² Test: 0.1209, Diff: 0.1524
Degree: 5 — R² Train: 0.2722, R² Test: -0.0311, Diff: 0.3033
